In [2]:
import praw
import pandas as pd
import os
import time
from datetime import datetime, timezone, timedelta

# 设置 Reddit API 凭证
reddit = praw.Reddit(
    client_id='Gd_KhZrVZVO79ou3LG_vVw',
    client_secret='-X_P63zpKewLKEreQJWXm2fFXjY1Ew',
    user_agent='reddit-crawler by /u/alpaca_1'
)

# === 参数设置 ===
subreddit_name = "SHIBArmy"           # 子版块名称
target_date = datetime(2025, 2, 20)             # 目标日期
next_date = target_date + timedelta(days=1)

start_timestamp = int(target_date.timestamp())
end_timestamp = int(next_date.timestamp())

# === 确保输出文件夹存在 ===
output_folder = "SHIBArmy_Posts"
os.makedirs(output_folder, exist_ok=True)

# === 收集帖子 ID ===
subreddit = reddit.subreddit(subreddit_name)
post_ids = []

print(f"🔍 正在查找 {target_date.date()} 当天的全部帖子...")
for submission in subreddit.new(limit=None):  # 获取最新帖子
    if start_timestamp <= submission.created_utc < end_timestamp:
        post_ids.append(submission.id)

print(f"✅ 找到 {len(post_ids)} 篇帖子")

# === 抓取评论 ===
rows = []

for pid in post_ids:
    try:
        submission = reddit.submission(id=pid)
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            rows.append({
                "post_id": pid,
                "author": str(comment.author) if comment.author else "N/A",
                "body": comment.body.replace("\n", " "),
                "score": comment.score,
                "created_utc": comment.created_utc,
                "date": datetime.fromtimestamp(comment.created_utc, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
            })
        time.sleep(1)
    except Exception as e:
        print(f"⚠️ 无法抓取帖子 {pid} 的评论: {e}")
        continue

# === 保存为 CSV（字段顺序固定）===
df = pd.DataFrame(rows, columns=["post_id", "author", "body", "score", "created_utc", "date"])
output_filename = os.path.join(output_folder, f"SHIBArmy_comments_{target_date.date()}.csv")
df.to_csv(output_filename, index=False, encoding="utf-8")
print(f"📁 已保存至：{output_filename}（共 {len(df)} 条评论）")

🔍 正在查找 2025-02-20 当天的全部帖子...
✅ 找到 2 篇帖子
📁 已保存至：SHIBArmy_Posts/SHIBArmy_comments_2025-02-20.csv（共 47 条评论）
